## Agenda

+ Introduction and setup environment (credentials)
+ Platform Setup and Walkthrough
+ Explanation of and Examination of Thunderbird Ground Truth
+ Model Selection ( 1 encoder/decoder), (2 decoder), eval against GPT4
+ Run experiment and show results
+ Evaluate results and discuss

## Foxfooding Introduction and Setup Environment

## Who we are, what we do, about the platform, etc. 


## Platform Setup and Walkthrough

In [2]:
# TODO Replace with ELB URL 
API_URL = "http://127.0.0.1/api/v1"

In [20]:
import requests
from typing import Dict, Any
import json

def make_request(
    url: str,
    method: str = "GET",
    params: Dict[str, Any] = None,
    data: Dict[str, Any] = None,
    headers: Dict[str, str] = None,
    timeout: int = 10
) -> requests.Response:
    """
    Make an HTTP request using the requests library.

    Args:
        url (str)
        method (str, optional): The HTTP method to use. Defaults to "GET".
        params (Dict[str, Any], optional): URL parameters to include in the request.
        data (Dict[str, Any], optional): Data to send in the request body.
        headers (Dict[str, str], optional): Headers to include in the request.
        timeout (int, optional): Timeout for the request in seconds. Defaults to 10.

    Returns:
        requests.Response: The response object from the request.

    Raises:
        requests.RequestException
    """
    try:
        response = requests.request(
            method=method.upper(),
            url=url,
            params=params,
            data=data,
            headers=headers,
            timeout=timeout
        )
        response.raise_for_status()
        print(f"{json.dumps(response.json(), indent = 2)}")
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        raise

In [ ]:
# Check a sample URL response
make_request(f"{API_URL}/health")

## Working with Datasets

In [22]:
# Show datasets to evaluate
make_request(f"{API_URL}/datasets")

{
  "total": 0,
  "items": []
}


In [ ]:
# Upload a dataset



In [ ]:
# Get information about a dataset

## Working with Experiments

## Creating a Ground Truth Run

In [24]:
# Show deployments

make_request(f"{API_URL}/ground-truth/deployments/")

{
  "total": 4,
  "items": [
    {
      "id": "83dde201-1a5e-41c4-bc3d-71ab48ccda36",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-11T15:45:56.098211Z",
      "updated_at": null
    },
    {
      "id": "cbb63174-b5f0-46f6-9ec1-aa7578ecf544",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-12T14:17:57.325988Z",
      "updated_at": null
    },
    {
      "id": "6955c1d4-2819-4178-8da8-821650f9f27e",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-12T14:18:50.715925Z",
      "updated_at": null
    },
    {
      "id": "19030812-2b05-4c39-a861-8d9e04aa0d47",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-12T15:02:07.229302Z",
      "updated_at": nul

In [ ]:
## A way to see all options on the platform 

In [9]:
payload =  """I recently added a beta release, version 7.0b1, of my extension, Clippings for Thunderbird, and selected the option to self-host it so that I can make it available to testers separately while regular users continue to see version 6.3.5, the current release version. However, the beta release is now incorrectly appearing in the Add-ons for Thunderbird public listing. URL to ATN listing: https://addons.thunderbird.net/en-US/thunderbird/addon/clippings-tb/. Thanks for the report, it is best to file an issue with the addons-server: https://github.com/thunderbird/addons-server I have not looked at it in detail, but self-hosted add-ons need an update_url entry in their manifest and that should prevent it from being accepted on ATN. Since we do not sign add-ons, self-hosted add-ons do not need to be submitted to ATN at all.My best-practice advice is:
- remove the beta version from ATN,
- create a dedicated branch/repo which holds the update information (either a branch in each add-on repo you want to self-host, or a single update-repo with the information for all your add-ons)
- host the XPI files either as an "beta" asset in a "github release", or directly as a file in the repo (I think the asset is the better choice),
- make sure that the manifest of your self-hosted XPI points to the correct update.json. John I've removed the beta version of my extension from ATN, and the listing for my extension now shows the current stable release. Thanks for your help!"""

In [14]:
requests.post(f"{API_URL}/ground-truth/deployments/19030812-2b05-4c39-a861-8d9e04aa0d47", json={"text": payload}).json()

{'text': {'result': 'Clippings for Thunderbird self-hosts a beta release of version 7.0b1. The beta release is now incorrectly appearing in the Add-ons for Thunderbirds public listing. John has removed the beta version from ATN, and the listing for my extension now shows the current stable release.'}}